In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy import stats

# Generate synthetic dataset for testing
np.random.seed(42)
n_samples = 100
data = {
    'income': np.random.normal(50000, 15000, n_samples),
    'credit_score': np.random.normal(650, 50, n_samples),
    'job_title': np.random.choice(['Engineer', 'Teacher', 'Doctor', 'Artist'], n_samples),
    'target': np.random.choice([0, 1], n_samples)
}

# Introduce missing values and outliers for testing
data['income'][np.random.randint(0, n_samples, 5)] = np.nan
data['credit_score'][np.random.randint(0, n_samples, 3)] = np.nan
data['income'][np.random.randint(0, n_samples, 2)] = 150000  # Outliers
df = pd.DataFrame(data)

In [2]:
df

income  credit_score job_title  target
0   57450.712295    579.231463   Teacher       0
1   47926.035482    628.967734   Teacher       0
2   59715.328072    632.864274  Engineer       0
3   72845.447846    609.886137    Artist       1
4   46487.699379    641.935714  Engineer       0
..           ...           ...       ...     ...
95  28047.275778    669.265869   Teacher       0
96  54441.804156    605.807128  Engineer       0
97  53915.829083    657.686255   Teacher       0
98  50076.701850    652.910436  Engineer       0
99  46481.192999    592.851485   Teacher       0

[100 rows x 4 columns]

In [3]:
# Handle missing values by filling with median
for column in df.select_dtypes(include=['float64', 'int64']).columns:
    df[column].fillna(df[column].median(), inplace=True)

/tmp/ipykernel_6101/2423168110.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(df[column].median(), inplace=True)


In [4]:
# Remove duplicates
df.drop_duplicates(inplace=True)

In [5]:
#Standardize numerical features 
scaler = StandardScaler()
numeric_features = df.select_dtypes(include=['float64', 'int64']).columns
df[numeric_features] = scaler.fit_transform(df[numeric_features])

In [6]:
# One-hot encode categorical features
df = pd.get_dummies(df, drop_first=True)

In [7]:
# Detect and remove outliers using Z-score
z_scores = np.abs(stats.zscore(df.select_dtypes(include=['float64', 'int64'])))
df = df[(z_scores < 3).all(axis=1)]

In [8]:
# Apply log transformation to skewed data
df['income_log'] = np.log1p(df['income'])

/anaconda/envs/new_ml_env/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [9]:
#Split data
X = df.drop('target', axis=1)
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)